In [1]:
import numpy as np
import os

from sklearn.utils import shuffle
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
face_data=[]
labels=[]
dataset_path='./data/'

class_id=0   #label for the given Title
names={}     #Mapping between id<->name

#Data Preparation
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        # Create a mapping between class_id and name
        names[class_id]=fx[:-4]
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #Create labels for the class
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        labels.append(target)
        
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(labels,axis=0).reshape((-1,1))
#trainset=np.concatenate((face_dataset,face_labels),axis=1)

In [3]:
X,Y=shuffle(face_dataset,face_labels,random_state=1)

In [4]:
X=X/255.0
X=X.reshape((-1,224,224,3))

In [5]:
Y=np_utils.to_categorical(Y)

In [6]:
print(X.shape)
print(Y.shape)

(20, 224, 224, 3)
(20, 2)


In [7]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model

from matplotlib import pyplot as plt 

In [8]:
model=ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

c:\users\masoo\appdata\local\programs\python\python37\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [9]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [10]:
# Global Average Pooling (GAP) Layer
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation='relu')(av1)
d1=Dropout(0.5)(fc1)
fc2=Dense(Y.shape[1],activation='softmax')(d1)

In [11]:
model_new=Model(input=model.input,outputs=fc2)

c:\users\masoo\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [12]:
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [13]:
# Compile
adam=Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [14]:
for ix in range(len(model_new.layers)):
    print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x000002AFF482DA48>
1 <keras.layers.convolutional.ZeroPadding2D object at 0x000002AFF482D308>
2 <keras.layers.convolutional.Conv2D object at 0x000002AFF482D748>
3 <keras.layers.normalization.BatchNormalization object at 0x000002AFF6D84308>
4 <keras.layers.core.Activation object at 0x000002AFF6F782C8>
5 <keras.layers.convolutional.ZeroPadding2D object at 0x000002AFF7034CC8>
6 <keras.layers.pooling.MaxPooling2D object at 0x000002AFF713FE08>
7 <keras.layers.convolutional.Conv2D object at 0x000002AFF7143E48>
8 <keras.layers.normalization.BatchNormalization object at 0x000002AFF714B908>
9 <keras.layers.core.Activation object at 0x000002AFF714BA88>
10 <keras.layers.convolutional.Conv2D object at 0x000002AFF714D2C8>
11 <keras.layers.normalization.BatchNormalization object at 0x000002AFF7169DC8>
12 <keras.layers.core.Activation object at 0x000002AFF8253A48>
13 <keras.layers.convolutional.Conv2D object at 0x000002AFF82631C8>
14 <keras.layers.conv

In [15]:
# Fine-Tuning (Here from layer 169)
for ix in range(169):
    model_new.layers[ix].trainable=False

In [16]:
# Compile
adam=Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [18]:
# Run on Google CoLab with epochs=10
model_new.fit(X,Y,shuffle=True,epochs=10,batch_size=4,validation_split=0.1)

Train on 18 samples, validate on 2 samples
Epoch 1/10
18/18 [==============================] - 49s 3s/step - loss: 0.5688 - accuracy: 0.6667 - val_loss: 0.6778 - val_accuracy: 0.5000
Epoch 2/10
18/18 [==============================] - 48s 3s/step - loss: 0.7700 - accuracy: 0.5556 - val_loss: 0.6773 - val_accuracy: 0.5000
Epoch 3/10
18/18 [==============================] - 49s 3s/step - loss: 0.3389 - accuracy: 0.8889 - val_loss: 0.6770 - val_accuracy: 0.5000
Epoch 4/10
18/18 [==============================] - 48s 3s/step - loss: 0.3168 - accuracy: 0.8889 - val_loss: 0.6771 - val_accuracy: 0.5000
Epoch 5/10
18/18 [==============================] - 49s 3s/step - loss: 0.3882 - accuracy: 0.7778 - val_loss: 0.6777 - val_accuracy: 0.5000
Epoch 6/10
18/18 [==============================] - 55s 3s/step - loss: 0.2884 - accuracy: 0.8333 - val_loss: 0.6777 - val_accuracy: 0.5000
Epoch 7/10
18/18 [==============================] - 57s 3s/step - loss: 0.1238 - accuracy: 1.0000 - val_loss: 0.6779 

In [19]:
with open("model.json", "w") as file:
    file.write(model_new.to_json())
model_new.save_weights("model.h5")

In [20]:
from pathlib import Path

In [21]:
count=0
idx_to_name={}
p=Path('data/')
dirs=p.glob('*')
for d in dirs:
    idx_to_name[count]=str(d).split('\\')[-1][:-4]
    count+=1

In [22]:
idx_to_name

{0: 'imran', 1: 'masoom'}